<a href="https://colab.research.google.com/github/jkraybill/gpt-2/blob/finetuning/GPT2-finetuning2-345M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To try out GPT-2, do this:

- go to the "Runtime" menu and click "Change runtime type" and make sure this is a Python 3 notebook, running with GPU hardware acceleration.
- use the "Files" section to the left to upload a text file called "corpus.txt" which contains all the text you want to train on.
- run the steps below in order.

In [0]:
import os
import json
import random
import re

In [2]:
!git clone https://github.com/jkraybill/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 208 (delta 0), reused 1 (delta 0), pack-reused 205
Receiving objects: 100% (208/208), 4.40 MiB | 14.58 MiB/s, done.
Resolving deltas: 100% (111/111), done.


In [3]:
cd gpt-2

/content/gpt-2


In [4]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 604kB 9.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.0.18 has requirement regex==2018.01.10, but you'll have regex 2017.4.5 which is incompatible.
  Found existing installation: regex 2018.1.10
    Uninstalling regex-2018.1.10:
      Successfully uninstalled regex-2018.1.10


In [5]:
!sh download_model.sh 345M

Fetching 345M/checkpoint
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    77  100    77    0     0   1375      0 --:--:-- --:--:-- --:--:--  1375
Fetching 345M/encoder.json
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1017k  100 1017k    0     0  17.1M      0 --:--:-- --:--:-- --:--:-- 17.1M
Fetching 345M/hparams.json
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    91  100    91    0     0   2527      0 --:--:-- --:--:-- --:--:--  2527
Fetching 345M/model.ckpt.data-00000-of-00001
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 135

The below step encodes your corpus into "NPZ" tokenized format for GPT-2.



In [6]:
!PYTHONPATH=src ./encode.py --in-text ../corpus-j3-v-easy-train.txt --out-npz train.txt.npz --model_name 345M
!PYTHONPATH=src ./encode.py --in-text ../corpus-j3-v-easy-val.txt --out-npz val.txt.npz --model_name 345M

Reading files
100% 1/1 [00:19<00:00, 19.22s/it]
Writing train.txt.npz
Reading files
100% 1/1 [00:07<00:00,  7.13s/it]
Writing val.txt.npz


Training is below. I usually get usable results with "stop_after" anywhere from 800 to 3000, but you can try going even higher. 800 steps takes only a few minutes.

"sample_every" controls how often you get sample output from the trained model.

"save_every" controls how often the model is saved.

"learning_rate" is the AI learning rate. 0.00005 is the rate I've gotten the best results with, but I think most people are running with significantly higher rates, so you could try adjusting it.

In [0]:
!PYTHONPATH=src ./trainval.py --dataset train.txt.npz --valset val.txt.npz --sample_every=1000 --save_every=1000 --learning_rate=0.00005 --stop_after=60000 --model_name=345M --batch_length=512

2019-05-10 11:47:36.671426: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-10 11:47:36.671779: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2cb2260 executing computations on platform Host. Devices:
2019-05-10 11:47:36.671834: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-10 11:47:36.930733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-10 11:47:36.931341: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2cb35a0 executing computations on platform CUDA. Devices:
2019-05-10 11:47:36.931392: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-10 11:47:36.931806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

The step below simply copies your trained model to the model directory, so the output will use your training. If you don't do this, you will be running against the trained GPT-2 model without your finetuning training.

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Run the below step to generate unconditional samples (i.e. "dream mode").

"top_k" controls how many options to consider per word (the larger, the more "diverse" the output - anything from 1 to about 50 usually works, I think values around 10 are pretty good).

"temperature" controls the sampling of the words, from 0 to 1 where 1 is the most "random".

"length" controls the number of words in each sample output.

This command will run continuously until you turn it off.

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 10 --temperature 1 --length=300 --model_name=345M

Traceback (most recent call last):
  File "src/generate_unconditional_samples.py", line 7, in <module>
    import tensorflow as tf
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 24, in <module>
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/__init__.py", line 52, in <module>
    from tensorflow.core.framework.graph_pb2 import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/core/framework/graph_pb2.py", line 6, in <module>
    from google.protobuf import descriptor as _descriptor
  File "/usr/local/lib/python3.6/dist-packages/google/protobuf/__init__.py", line 37, in <module>
    __import__('pkg_resources').declare_namespace(__name__)
  File "/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py", line 3241, in <module>
    @_call_aside
  File "/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py", line 3225, in _call_aside
    f(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/pkg_resource

Run the command below to run in interactive / "completion" mode. You will get a prompt; just type in whatever prompt text you want, and the model will attempt to complete it "nsamples" times.

"top_k", "length", and "temperature" work as specified above.

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 1 --length=30 --temperature 0.1 --nsamples 3 --model_name=345M

2019-05-08 17:53:50.739097: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-08 17:53:50.739382: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1f36520 executing computations on platform Host. Devices:
2019-05-08 17:53:50.739454: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-08 17:53:50.972419: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-08 17:53:50.972910: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1f35e40 executing computations on platform CUDA. Devices:
2019-05-08 17:53:50.972939: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-08 17:53:50.973272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de